In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization,GRU, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, LearningRateScheduler
from tensorflow.keras.metrics import Recall
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import precision_score, recall_score,accuracy_score
from tensorflow.keras.initializers import HeUniform
from tensorflow.keras import regularizers
import os
import sklearn.metrics
import tensorflow as tf
from sklearn.metrics import confusion_matrix
import seaborn as sns
import keras as ks
from sklearn.preprocessing import MinMaxScaler
from keras import layers
from tensorflow.keras.models import Model


In [ ]:
dataset1 = pd.read_csv('/content/drive/MyDrive/oslo4144_feature_eng.csv', index_col='ts', parse_dates=True)
dataset2 = pd.read_csv('/content/drive/MyDrive/oslo4147_feature_eng.csv', index_col='ts', parse_dates=True)
dataset3 = pd.read_csv('/content/drive/MyDrive/gelio_feature_eng.csv', index_col='ts', parse_dates=True)
dataset4 = pd.read_csv('/content/drive/MyDrive/tromso_feature_eng.csv', index_col='ts', parse_dates=True)
dataset5 = pd.read_csv('/content/drive/MyDrive/bergen4126_feature_eng.csv', index_col='ts', parse_dates=True)


In [ ]:
dataset6 = pd.read_csv('/content/drive/MyDrive/trond7065_feature_eng.csv', index_col='ts', parse_dates=True)

In [ ]:
dataset6 = pd.read_csv('/content/drive/MyDrive/bud_feature_eng.csv', index_col='ts', parse_dates=True)

In [ ]:
dataset6

,rtt,event_rssi,event_rsrp,event_rsrq,event_band,event_device_state,event_lte_freq,anomaly_label,day_of_week_Friday,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday
ts,,,,,,,,,,,,,,,
2023-11-01 00:00:00,0.051132,-80.0,-112.0,-12.0,3.0,3.0,1800.0,1,False,False,False,False,False,False,True
2023-11-01 00:00:01,0.050266,-80.0,-111.0,-12.0,3.0,3.0,1800.0,1,False,False,False,False,False,False,True
2023-11-01 00:00:02,0.048981,-80.0,-111.0,-12.0,3.0,3.0,1800.0,1,False,False,False,False,False,False,True
2023-11-01 00:00:03,0.047081,-80.0,-111.0,-12.0,3.0,3.0,1800.0,1,False,False,False,False,False,False,True
2023-11-01 00:00:04,0.047388,-80.0,-111.0,-12.0,3.0,3.0,1800.0,1,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-31 23:59:55,0.052416,-71.0,-104.0,-15.0,20.0,3.0,800.0,1,False,False,False,True,False,False,False
2023-12-31 23:59:56,0.051409,-71.0,-104.0,-15.0,20.0,3.0,800.0,1,False,False,False,True,False,False,False
2023-12-31 23:59:57,0.052315,-71.0,-104.0,-16.0,20.0,3.0,800.0,1,False,False,False,True,False,False,False


In [ ]:
dataset1

In [ ]:
# Select data for one week (adjust the date range as needed)
start_date = '2023-12-01'
end_date = '2023-12-08'

dataset1_week = dataset1.loc[start_date:end_date]
dataset2_week = dataset2.loc[start_date:end_date]
dataset3_week = dataset3.loc[start_date:end_date]
dataset4_week = dataset4.loc[start_date:end_date]
dataset5_week = dataset5.loc[start_date:end_date]
dataset6_week = dataset6.loc[start_date:end_date]

# Drop columns you don't need
columns_to_drop = ['day_of_week_Friday', 'day_of_week_Monday','day_of_week_Saturday','day_of_week_Thursday','day_of_week_Sunday','day_of_week_Tuesday','day_of_week_Wednesday']
dataset1_week.drop(columns=columns_to_drop, inplace=True)
dataset2_week.drop(columns=columns_to_drop, inplace=True)
dataset3_week.drop(columns=columns_to_drop, inplace=True)
dataset4_week.drop(columns=columns_to_drop, inplace=True)
dataset5_week.drop(columns=columns_to_drop, inplace=True)
dataset6_week.drop(columns=columns_to_drop, inplace=True)

<ipython-input-7-d885f6bd12b4>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset1_week.drop(columns=columns_to_drop, inplace=True)
<ipython-input-7-d885f6bd12b4>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset2_week.drop(columns=columns_to_drop, inplace=True)
<ipython-input-7-d885f6bd12b4>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset3_week.drop(columns=columns_to_drop, inplace=True)
<ipython-input-7-d885f

In [ ]:
dataset1_week

In [ ]:
# Step 1: Calculate rolling statistics for each dataset
window_size = 10
for dataset in [dataset1_week, dataset2_week, dataset3_week, dataset4_week, dataset5_week,dataset6_week]:
    dataset['rtt_mean'] = dataset['rtt'].rolling(window=window_size).mean()
    dataset['rtt_std'] = dataset['rtt'].rolling(window=window_size).std()
    dataset['rtt_max'] = dataset['rtt'].rolling(window=window_size).max()
    dataset['rtt_min'] = dataset['rtt'].rolling(window=window_size).min()

# Step 2: Add hour and day_of_week features to each dataset
for dataset in [dataset1_week, dataset2_week, dataset3_week, dataset4_week, dataset5_week,dataset6_week]:
    dataset['hour'] = dataset.index.hour
    dataset['day_of_week'] = dataset.index.dayofweek


In [ ]:
for dataset in [dataset1_week, dataset2_week, dataset3_week, dataset4_week, dataset5_week,dataset6_week]:
      dataset.dropna(inplace=True)

<ipython-input-11-5729fc15eb25>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.dropna(inplace=True)


In [ ]:
dataset1_week.to_csv('/content/drive/MyDrive/4144.csv')
dataset2_week.to_csv('/content/drive/MyDrive/4147.csv')
dataset3_week.to_csv('/content/drive/MyDrive/4135.csv')
dataset4_week.to_csv('/content/drive/MyDrive/4137.csv')

In [ ]:
dataset6_week.to_csv('/content/drive/MyDrive/4152_good.csv')

In [ ]:
dataset5_week.to_csv('/content/drive/MyDrive/4126.csv')

In [ ]:
dataset6_week

,rtt,event_rssi,event_rsrp,event_rsrq,event_band,event_device_state,event_lte_freq,anomaly_label,rtt_mean,rtt_std,rtt_max,rtt_min,hour,day_of_week
ts,,,,,,,,,,,,,,
2023-12-01 00:00:09,0.063362,-74.0,-104.0,-11.0,20.0,2.0,800.0,1,0.054884,0.008615,0.066448,0.039687,0,4
2023-12-01 00:00:10,0.062268,-74.0,-104.0,-11.0,20.0,2.0,800.0,1,0.055868,0.008862,0.066448,0.039687,0,4
2023-12-01 00:00:11,0.062191,-74.0,-104.0,-11.0,20.0,2.0,800.0,1,0.056935,0.008923,0.066448,0.039687,0,4
2023-12-01 00:00:12,0.060174,-74.0,-104.0,-11.0,20.0,2.0,800.0,1,0.058983,0.006563,0.066448,0.046373,0,4
2023-12-01 00:00:13,0.059134,-76.0,-103.0,-11.0,20.0,2.0,800.0,1,0.059945,0.005664,0.066448,0.046373,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-08 23:59:55,0.056170,-72.0,-104.0,-14.0,20.0,3.0,800.0,1,0.057833,0.006794,0.066304,0.043421,23,4
2023-12-08 23:59:56,0.076090,-72.0,-104.0,-14.0,20.0,3.0,800.0,1,0.058812,0.008611,0.076090,0.043421,23,4
2023-12-08 23:59:57,0.065072,-72.0,-104.0,-14.0,20.0,3.0,800.0,1,0.058795,0.008597,0.076090,0.043421,23,4


In [ ]:
neg, pos = np.bincount(dataset6_week['anomaly_label'])
total = neg + pos
weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)
adjusted_weight_for_0 = weight_for_0 * 1.0  # Example: decrease if too many false positives
adjusted_weight_for_1 = weight_for_1 * 1.0  # Example: increase if too many false negatives

class_weight = {0: adjusted_weight_for_0, 1: adjusted_weight_for_1}

In [ ]:
neg

118964

In [ ]:
pos

567814